In [6]:
import pandas as pd
import numpy as np
import sys
sys.path.append("..")
from data.scripts.simplified_finance_stats.fin_stats import fin_stats
from data.scripts.simplified_finance_stats.fin_stats_2 import fin_stats_2
from data.scripts.simplified_finance_stats.stock_stats import stock_stats
from value import valuation
from input_params import get_inp_params
from sklearn.linear_model import LinearRegression
from input_params import get_inp_params

import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
# Set path for data
base_path = '../data/'
sheets_path = 'combined_simplified/combined_all_us.csv'
other_path = 'combined_simplified/others_all_us.csv'
mkt_path = 'combined_simplified/stock_stats_all_us.csv'

# setup all data
finances = fin_stats(base_path + sheets_path)
fin_others = fin_stats_2(base_path + other_path)
mkt_data = stock_stats(base_path + mkt_path)

Total Missing tickers: 181
Total Missing tickers: 181
Total Missing tickers: 181


In [7]:
# Get the fundamental data
tick = 'FB'
b = finances.get_sheet(tick,"balance_sheet")
i = finances.get_sheet(tick,"income_sheet")
c = finances.get_sheet(tick,"cashflow_sheet")
o = fin_others.get_sheet(tick)
mk = mkt_data.get_stock_data(tick)

inp_params = get_inp_params(tick,finances,fin_others,mkt_data)

In [23]:

def revenue_projection(hist_period = 20, time_period=5, order=2,reduction_factor=0.75):
    """Returns the projected revenue for 2*time_period years
        For 2nd order: Growth rate constant for the first half and
        decreases linearly to rf rate"""

    if order == 2:
        hist_rev  = i.loc['revt'].values
        hist_rev = hist_rev[-20::]
        
        hist_period = np.array(i.columns)[-20::]

        past_N = float(hist_period.shape[0])

        CAGR = ((hist_rev[-1]/hist_rev[0])**(1/past_N) - 1)*reduction_factor
        
        # First half
        p = range(1,time_period+1)
        factor = [(1+CAGR)**k for k in p]

        # Second half
        rf_rate = inp_params['rf_rate']
        grw_rate = np.linspace(CAGR,rf_rate,time_period+1)[1::]
        for r in grw_rate:
            factor.append((1+r)*factor[-1])

        rev = hist_rev[-1]*np.array(factor)
        return rev

In [24]:
revenue_projection()

0.30204930964496857
[1.3020493096449686, 1.6953324047469394, 2.2074063872194967, 2.87415196258504, 3.7422875786985825, 4.664572285663201, 5.554803974255456, 6.306089677858758, 6.8083688500120605, 6.9721101208548495]


array([ 35986.03881997,  46855.5970024 ,  61008.29772997,  79435.81194193,
       103429.34410007, 128919.44883116, 153523.67224047, 174287.70651666,
       188169.69827663, 192695.17952019])